In [1]:
import pandas as pd

class Doc():
    def __init__(self, num_links: int):
        self.num_links = num_links
        self.page_rank = 1
        self.children = list()
        self.page_ranks = [1]
        
    def calc_page_rank(self, d: float = 0.1):
        pr = d + (1 - d) * sum(doc.page_rank/doc.num_links for doc in self.children)
        self.page_rank = pr
        self.page_ranks.append(pr)
        
A = Doc(2)
B = Doc(1)
C = Doc(1)

A.children = [B, C]
B.children = [A]
C.children = [A]

for _ in range(25):
    A.calc_page_rank()
    B.calc_page_rank()
    C.calc_page_rank()
    
df = pd.DataFrame(data={'PageRankA': A.page_ranks,
                    'PageRankB': B.page_ranks,
                    'PageRankC': C.page_ranks,})

df.index.name = 'Iteration'

df

,PageRankA,PageRankB,PageRankC
Iteration,,,
0,1.000000,1.000000,1.000000
1,1.900000,0.955000,0.955000
2,1.819000,0.918550,0.918550
3,1.753390,0.889026,0.889026
4,1.700246,0.865111,0.865111
5,1.657199,0.845740,0.845740
6,1.622331,0.830049,0.830049
7,1.594088,0.817340,0.817340
8,1.571212,0.807045,0.807045


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [53]:
raw_sentences = ["Jeff purchased a new tractor, so he could fix the grade on his gravel road.", 
                 "Professor Jurkat gave me a perfect grade on this homework assignment."]

sentences = [sent.lower().replace(',', '').replace('.', '') for sent in raw_sentences]
sentences

['jeff purchased a new tractor so he could fix the grade on his gravel road',
 'professor jurkat gave me a perfect grade on this homework assignment']

In [115]:
senses = ['a body of students who are taught together  early morning classes are always sleepy',
 'a relative position or degree of value in a graded group  lumber of the highest grade',
 'the gradient of a slope or road or other surface  the road had a steep grade',
 'one-hundredth of a right angle',
 'a degree of ablaut',
 'a number or letter indicating quality she made good marks in algebra grade A milk what was your score on your homework',
 'the height of the ground on which something stands  the base of the tower was below grade',
 'a position on a scale of intensity or amount or quality  a moderate grade of intelligence  a high level of care is required  it is all a matter of degree',
 'a variety of cattle produced by crossbreeding with a superior breed',
          
 'assign a rank or rating to  how would you rank these students  The restaurant is rated highly in the food guide',
 'level to the right gradient',
 'assign a grade or rank to, according to ones evaluation  grade tests  score the SAT essays  mark homework',
 'determine the grade of or assign a grade to']


In [116]:
dictionary = set(' '.join(senses).lower().split())
dictionary.update(' '.join(sentences).split())

In [117]:
dict((Counter(sentences[0].split())))

{'jeff': 1,
 'purchased': 1,
 'a': 1,
 'new': 1,
 'tractor': 1,
 'so': 1,
 'he': 1,
 'could': 1,
 'fix': 1,
 'the': 1,
 'grade': 1,
 'on': 1,
 'his': 1,
 'gravel': 1,
 'road': 1}

In [118]:
sentence_rows = [dict((Counter(sent.split()))) for sent in sentences]
sense_rows = [dict((Counter(sense.split()))) for sense in senses]

In [119]:
df = pd.DataFrame(data=sentence_rows+sense_rows).fillna(0)

df.head()

,jeff,purchased,a,new,tractor,so,he,could,fix,the,...,guide,"to,",according,ones,evaluation,tests,SAT,essays,mark,determine
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
norm = (df*df).sum(axis=1).pow(0.5)

norm

0     3.872983
1     3.316625
2     4.000000
3     4.472136
4     4.898979
5     2.236068
6     2.000000
7     4.898979
8     5.567764
9     8.124038
10    3.605551
11    4.795832
12    2.236068
13    4.472136
14    3.316625
dtype: float64

In [121]:
norm_df = df.divide(norm, axis=0)
norm_df.head()

,jeff,purchased,a,new,tractor,so,he,could,fix,the,...,guide,"to,",according,ones,evaluation,tests,SAT,essays,mark,determine
0,0.258199,0.258199,0.258199,0.258199,0.258199,0.258199,0.258199,0.258199,0.258199,0.258199,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.301511,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.447214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.223607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
def cos_sim(s1, s2):
    return (s1 * s2).sum()

In [123]:
sent = norm_df.iloc[0]

for n, sense in norm_df.iloc[2:].iterrows():
    print(cos_sim(sent, sense))

0.06454972243679027
0.23094010767585027
0.3689323936863109
0.11547005383792514
0.12909944487358055
0.15811388300841897
0.27824333745610097
0.22247460415730483
0.14322297480788657
0.10767638041163309
0.11547005383792514
0.23094010767585027
0.3113995776646092


In [113]:
sent = norm_df.iloc[1]

for n, sense in norm_df.iloc[2:].iterrows():
    print(cos_sim(sent, sense))

0.07537783614444091
0.20225995873897262
0.18463723646899913
0.13483997249264842
0.15075567228888181
0.2461829819586655
0.10830607221477648
0.2597943666588219
0.16724840200141816
0.0
0.06286946134619315
0.0
0.26967994498529685
0.2727272727272727


In [114]:
s1 * s2

jeff         0.00000
purchased    0.00000
a            0.11547
new          0.00000
tractor      0.00000
              ...   
score        0.00000
SAT          0.00000
essays       0.00000
mark         0.00000
determine    0.00000
Length: 118, dtype: float64

In [15]:
raw = """1. class, form, grade, course -- (a body of students who are taught together; "early morning classes are always sleepy")
2. grade, level, tier -- (a relative position or degree of value in a graded group; "lumber of the highest grade")
3. grade -- (the gradient of a slope or road or other surface; "the road had a steep grade")
4. grad, grade -- (one-hundredth of a right angle)
5. grade, gradation -- (a degree of ablaut)
6. mark, grade, score -- (a number or letter indicating quality (especially of a student's performance); "she made good marks in algebra"; "grade A milk"; "what was your score on your homework?")
7. grade, ground level -- (the height of the ground on which something stands; "the base of the tower was below grade")
8. degree, grade, level -- (a position on a scale of intensity or amount or quality; "a moderate grade of intelligence"; "a high level of care is required"; "it is all a matter of degree")
9. grade -- (a variety of cattle produced by crossbreeding with a superior breed)

Overview of verb grade

The verb grade has 4 senses (first 1 from tagged texts)
                                          
1. (assign a rank or rating to; "how would you rank these students?"; "The restaurant is rated highly in the food guide")
2. grade -- (level to the right gradient)
3. grade, score, mark -- (assign a grade or rank to, according to one's evaluation; "grade tests"; "score the SAT essays"; "mark homework")
4. grade -- (determine the grade of or assign a grade to)
"""

In [35]:
senses = list()
for line in raw.split('\n'):
    if '(' in line and ')' in line:
        sense = line.split('(')[1].split(')')[0]
        senses.append(sense)

In [36]:
senses

['a body of students who are taught together; "early morning classes are always sleepy"',
 'a relative position or degree of value in a graded group; "lumber of the highest grade"',
 'the gradient of a slope or road or other surface; "the road had a steep grade"',
 'one-hundredth of a right angle',
 'a degree of ablaut',
 'a number or letter indicating quality ',
 'the height of the ground on which something stands; "the base of the tower was below grade"',
 'a position on a scale of intensity or amount or quality; "a moderate grade of intelligence"; "a high level of care is required"; "it is all a matter of degree"',
 'a variety of cattle produced by crossbreeding with a superior breed',
 'first 1 from tagged texts',
 'assign a rank or rating to; "how would you rank these students?"; "The restaurant is rated highly in the food guide"',
 'level to the right gradient',
 'assign a grade or rank to, according to one\'s evaluation; "grade tests"; "score the SAT essays"; "mark homework"',
 

In [32]:
seg.split(')')

['determine the grade of or assign a grade to', '']

In [12]:
next(i)

'Sense'

In [21]:
A.page_ranks

[1,
 1.9000000000000001,
 1.8595000000000002,
 1.7861950000000002,
 1.7268179500000003,
 1.6787225395000005,
 1.6397652569950005,
 1.6082098581659505,
 1.58264998511442,
 1.5619464879426803,
 1.545176655233571,
 1.5315930907391926,
 1.520590403498746,
 1.5116782268339846,
 1.5044593637355277,
 1.4986120846257776,
 1.49387578854688,
 1.490039388722973,
 1.4869319048656082,
 1.4844148429411428,
 1.4823760227823257,
 1.480724578453684,
 1.4793869085474842,
 1.4783033959234624,
 1.4774257506980046,
 1.4767148580653837,
 1.4761390350329608]

In [22]:
B.page_ranks

[1,
 0.9550000000000001,
 0.936775,
 0.9037877500000001,
 0.8770680775000002,
 0.8554251427750003,
 0.8378943656477502,
 0.8236944361746777,
 0.812192493301489,
 0.8028759195742061,
 0.795329494855107,
 0.7892168908326367,
 0.7842656815744358,
 0.7802552020752931,
 0.7770067136809875,
 0.7743754380815999,
 0.7722441048460961,
 0.7705177249253379,
 0.7691193571895237,
 0.7679866793235143,
 0.7670692102520466,
 0.7663260603041578,
 0.7657241088463679,
 0.7652365281655581,
 0.764841587814102,
 0.7645216861294226,
 0.7642625657648324]